<center><h1> ETI195 - Etica para Ciencia de datos y Estadistica </h1><center>
<center><h2> Lab Sumativo 2: Fairness <h2></center>

## Contexto:

El set de datos llamado **"adult dataset"** o **"census income dataset"** contiene datos demográficos y financieros de personas, como su edad, nivel educativo, estado civil, ocupación, relación familiar, entre otros. Su principal finalidad es determinar si una persona percibe un ingreso anual de más o menos de $50,000, siendo así un problema de clasificación binaria. Este conjunto de datos se considera delicado debido a que contiene información personal e ingresos, por lo que regularmente se emplea para enseñar y poner en práctica procesos de análisis de datos de forma ética y responsable.

En la presente tarea se entrega la versión de este dataset disponible en [AIF360](https://aif360.readthedocs.io/en/stable/modules/generated/aif360.datasets.AdultDataset.html#aif360.datasets.AdultDataset), con algunas modificaciones. El objetivo es evaluar la implementación de los tooltkits Aequitas y AIF360 en el contexto de este dataset.
## Instrucciones:

- La siguiente tarea es **individual**.
- Fecha de entrega: Lunes 13 de mayo, en Canvas.

In [1]:
import pandas as pd
import numpy as np


In [2]:

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score, roc_curve

In [50]:
from aif360.sklearn.datasets import fetch_german
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import ClassificationMetric


In [4]:
df = pd.read_csv('adult_data.csv')
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,34.0,Private,10th,6.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,30.0,United-States,<=50K


### Pregunta 1: Realice el pre-procesamiento de los datos e implemente un modelo de clasificación (Regresión Logística). Obtenga el accuracy del modelo sobre los datos de testeo (1 pto).

#### Indicaciones adicionales:

- Recuerde utilizar random_state o seeds según sea necesario para la reproducibilidad de sus resultados.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45222 entries, 0 to 45221
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             45222 non-null  float64
 1   workclass       45222 non-null  object 
 2   education       45222 non-null  object 
 3   education-num   45222 non-null  float64
 4   marital-status  45222 non-null  object 
 5   occupation      45222 non-null  object 
 6   relationship    45222 non-null  object 
 7   race            45222 non-null  object 
 8   sex             45222 non-null  object 
 9   capital-gain    45222 non-null  float64
 10  capital-loss    45222 non-null  float64
 11  hours-per-week  45222 non-null  float64
 12  native-country  45222 non-null  object 
 13  income          45222 non-null  object 
dtypes: float64(5), object(9)
memory usage: 4.8+ MB


In [6]:
# Binarizacion de algunas variables categoricas para poder trabajar con ellas en el modelo de regresion logistica 
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

df

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,1,0.0,0.0,40.0,United-States,0
1,38.0,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,1,0.0,0.0,50.0,United-States,0
2,28.0,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,1,0.0,0.0,40.0,United-States,1
3,44.0,Private,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,1,7688.0,0.0,40.0,United-States,1
4,34.0,Private,10th,6.0,Never-married,Other-service,Not-in-family,White,1,0.0,0.0,30.0,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,27.0,Private,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,0,0.0,0.0,38.0,United-States,0
45218,40.0,Private,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,1,0.0,0.0,40.0,United-States,1
45219,58.0,Private,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,0,0.0,0.0,40.0,United-States,0
45220,22.0,Private,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,1,0.0,0.0,20.0,United-States,0


In [7]:
# Separamos x e y
X = df.loc[:, df.columns != 'income']
y = df.loc[:, df.columns == 'income']


In [8]:
df1 = X.copy()


In [9]:
# Obtenemos variables dummies
ignore = ['sex']

for col in df1.select_dtypes(include='object').columns:
    if col in ignore:
        continue
    else:
        dummies = pd.get_dummies(X[col])
        X = pd.concat([X, dummies], axis=1).drop(columns=[col])
X.head()

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,Federal-gov,Local-gov,Private,Self-emp-inc,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,25.0,7.0,1,0.0,0.0,40.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
1,38.0,9.0,1,0.0,0.0,50.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2,28.0,12.0,1,0.0,0.0,40.0,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
3,44.0,10.0,1,7688.0,0.0,40.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
4,34.0,6.0,1,0.0,0.0,30.0,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False


In [10]:
# Cambiamos false por 0 y true por 1
X = X.replace({False: 0, True: 1})
X.head()

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,Federal-gov,Local-gov,Private,Self-emp-inc,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
0,25.0,7.0,1,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,38.0,9.0,1,0.0,0.0,50.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,28.0,12.0,1,0.0,0.0,40.0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44.0,10.0,1,7688.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,34.0,6.0,1,0.0,0.0,30.0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
# Datos de entrenamiento y test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=908)


In [12]:
# Sacamos la edad
X_train.drop(columns=['age'], inplace=True)
X_test.drop(columns=['age'], inplace=True)


In [13]:
df_train = X_train.copy()
df_train['income'] = y_train


In [14]:
# Discriminacion por sexo 

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]


In [15]:
# instanciamos binarylabeldataset
attribute_params = dict(protected_attribute_names=['sex'], label_names=['income'])
dt_train = BinaryLabelDataset(df=df_train, **attribute_params)


In [16]:
# Finalmente, la instancia de BinaryLabelDatasetMetric con la edad como atributo protegido.

metric_age_train = BinaryLabelDatasetMetric(dt_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

In [17]:
print(f'Train Statistical Parity Difference (age): {metric_age_train.mean_difference()}')

Train Statistical Parity Difference (age): -0.1975830166220287


In [18]:
lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908, max_iter=1000)
lr.fit(X_train, np.ravel(y_train))

LogisticRegressionCV(cv=10, max_iter=1000, random_state=908, solver='liblinear')

In [19]:
y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred = y_pred_proba >= 0.5

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred)}")


Accuracy: 0.8437810945273632
Recall: 0.5729632945389436
F1: 0.6443493581676315


### Pregunta 2: Evalúe las clasificaciones del modelo de la pregunta anterior a través de las métricas de Fairness facilitadas en Aequitas.

In [20]:
from aequitas.group import Group

In [21]:
# Creare una columna de age_cat,  con greater than 45, 25- 45 y Less than 45
df['age_cat'] = pd.cut(df['age'], bins=[0, 25, 45, 100], labels=['Less than 25', '25-45', 'Greater than 45'])

df

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,age_cat
0,25.0,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,1,0.0,0.0,40.0,United-States,0,Less than 25
1,38.0,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,1,0.0,0.0,50.0,United-States,0,25-45
2,28.0,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,1,0.0,0.0,40.0,United-States,1,25-45
3,44.0,Private,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,1,7688.0,0.0,40.0,United-States,1,25-45
4,34.0,Private,10th,6.0,Never-married,Other-service,Not-in-family,White,1,0.0,0.0,30.0,United-States,0,25-45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,27.0,Private,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,0,0.0,0.0,38.0,United-States,0,25-45
45218,40.0,Private,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,1,0.0,0.0,40.0,United-States,1,25-45
45219,58.0,Private,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,0,0.0,0.0,40.0,United-States,0,Greater than 45
45220,22.0,Private,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,1,0.0,0.0,20.0,United-States,0,Less than 25


In [22]:
# Seleccionamos las columnas necesarias

df_aequitas = df[['income', 'sex', 'race', 'age_cat', 'education-num']]
df_aequitas

,income,sex,race,age_cat,education-num
0,0,1,Black,Less than 25,7.0
1,0,1,White,25-45,9.0
2,1,1,White,25-45,12.0
3,1,1,Black,25-45,10.0
4,0,1,White,25-45,6.0
...,...,...,...,...,...
45217,0,0,White,25-45,12.0
45218,1,1,White,25-45,9.0
45219,0,0,White,Greater than 45,9.0
45220,0,1,White,Less than 25,9.0


In [23]:
df_aequitas.rename(columns={'sex': 'score', 'income': 'label_value'}, inplace=True)


C:\Users\Poble\AppData\Local\Temp\ipykernel_4940\21422225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aequitas.rename(columns={'sex': 'score', 'income': 'label_value'}, inplace=True)


In [24]:
df_aequitas.head()

,label_value,score,race,age_cat,education-num
0,0,1,Black,Less than 25,7.0
1,0,1,White,25-45,9.0
2,1,1,White,25-45,12.0
3,1,1,Black,25-45,10.0
4,0,1,White,25-45,6.0


In [25]:

df_aequitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45222 entries, 0 to 45221
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   label_value    45222 non-null  int64   
 1   score          45222 non-null  int64   
 2   race           45222 non-null  object  
 3   age_cat        45222 non-null  category
 4   education-num  45222 non-null  float64 
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 1.4+ MB


In [26]:
# Cambiamos el tipo de la columna age_cat a object 
df_aequitas['age_cat'] = df_aequitas['age_cat'].astype('object')
df_aequitas['education-num'] = df_aequitas['education-num'].astype('object')

df_aequitas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45222 entries, 0 to 45221
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   label_value    45222 non-null  int64 
 1   score          45222 non-null  int64 
 2   race           45222 non-null  object
 3   age_cat        45222 non-null  object
 4   education-num  45222 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.7+ MB


C:\Users\Poble\AppData\Local\Temp\ipykernel_4940\3947863477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aequitas['age_cat'] = df_aequitas['age_cat'].astype('object')
C:\Users\Poble\AppData\Local\Temp\ipykernel_4940\3947863477.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aequitas['education-num'] = df_aequitas['education-num'].astype('object')


In [27]:
group = Group()
xtab, _ = group.get_crosstabs(df_aequitas)
xtab.head()

,model_id,score_threshold,k,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,...,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,30527,race,Amer-Indian-Eskimo,0.439080,0.735849,0.397906,0.084337,0.855019,...,0.618391,230,14,152,39,53,382,435,45222,0.121839
1,0,binary 0/1,30527,race,Asian-Pac-Islander,0.518035,0.823848,0.397216,0.149083,0.649366,...,0.665388,563,65,371,304,369,934,1303,45222,0.283193
2,0,binary 0/1,30527,race,Black,0.559603,0.764045,0.530049,0.060461,0.809701,...,0.507096,1736,126,1958,408,534,3694,4228,45222,0.126301
3,0,binary 0/1,30527,race,Other,0.433428,0.800000,0.379870,0.071429,0.841410,...,0.643059,191,9,117,36,45,308,353,45222,0.127479
4,0,binary 0/1,30527,race,White,0.493021,0.857451,0.363396,0.122444,0.676092,...,0.694548,18268,1455,10428,8752,10207,28696,38903,45222,0.262371


In [28]:
xtab.columns

Index(['model_id', 'score_threshold', 'k', 'attribute_name', 'attribute_value',
       'accuracy', 'tpr', 'tnr', 'for', 'fdr', 'fpr', 'fnr', 'npv',
       'precision', 'pp', 'pn', 'ppr', 'pprev', 'fp', 'fn', 'tn', 'tp',
       'group_label_pos', 'group_label_neg', 'group_size', 'total_entities',
       'prev'],
      dtype='object')

In [29]:
absolute_metrics = group.list_absolute_metrics(xtab)
absolute_metrics

['accuracy',
 'tpr',
 'tnr',
 'for',
 'fdr',
 'fpr',
 'fnr',
 'npv',
 'precision',
 'ppr',
 'pprev',
 'prev']

In [30]:
absolute_xtab = xtab[['attribute_name', 'attribute_value'] + absolute_metrics]
absolute_xtab

,attribute_name,attribute_value,accuracy,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,race,Amer-Indian-Eskimo,0.439080,0.735849,0.397906,0.084337,0.855019,0.602094,0.264151,0.915663,0.144981,0.008812,0.618391,0.121839
1,race,Asian-Pac-Islander,0.518035,0.823848,0.397216,0.149083,0.649366,0.602784,0.176152,0.850917,0.350634,0.028401,0.665388,0.283193
2,race,Black,0.559603,0.764045,0.530049,0.060461,0.809701,0.469951,0.235955,0.939539,0.190299,0.070233,0.507096,0.126301
3,race,Other,0.433428,0.800000,0.379870,0.071429,0.841410,0.620130,0.200000,0.928571,0.158590,0.007436,0.643059,0.127479
4,race,White,0.493021,0.857451,0.363396,0.122444,0.676092,0.636604,0.142549,0.877556,0.323908,0.885118,0.694548,0.262371
5,age_cat,25-45,0.481271,0.834465,0.351380,0.147668,0.678825,0.648620,0.165535,0.852332,0.321175,0.538834,0.698590,0.268878
6,age_cat,Greater than 45,0.563959,0.878023,0.390212,0.147436,0.556615,0.609788,0.121977,0.852564,0.443385,0.305795,0.705327,0.356177
7,age_cat,Less than 25,0.446511,0.717791,0.441169,0.012439,0.975332,0.558831,0.282209,0.987561,0.024668,0.155371,0.561900,0.019311
8,education-num,1.0,0.291667,1.000000,0.281690,0.000000,0.980769,0.718310,0.000000,1.000000,0.019231,0.001703,0.722222,0.013889
9,education-num,2.0,0.288288,1.000000,0.261682,0.000000,0.951807,0.738318,0.000000,1.000000,0.048193,0.005438,0.747748,0.036036


In [31]:
xtab.shape

(24, 27)

### Pregunta 3: Aplique el algoritmo EqOddsPostProcessing sobre las predicciones del modelo de la pregunta 1. Obtenga las predicciones modificadas y evalúelas. ¿Qué cambios hubo en las métricas de Fairness? (2 ptos).

#### Indicaciones adicionales:

- Obtenga tanto las intancias de BinaryLabelDatasetMetric como de ClassificationMetric y calcule al menos una métrica de Fairness.


In [32]:
X_test

,education-num,sex,capital-gain,capital-loss,hours-per-week,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
24938,13.0,1,0.0,0.0,30.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
22771,13.0,1,4386.0,0.0,47.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
6957,14.0,0,0.0,0.0,45.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
23546,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
18142,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29934,10.0,0,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
10608,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
32133,5.0,1,0.0,0.0,35.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
41809,9.0,1,0.0,0.0,45.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
X_test

,education-num,sex,capital-gain,capital-loss,hours-per-week,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
24938,13.0,1,0.0,0.0,30.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
22771,13.0,1,4386.0,0.0,47.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
6957,14.0,0,0.0,0.0,45.0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
23546,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
18142,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29934,10.0,0,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
10608,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
32133,5.0,1,0.0,0.0,35.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
41809,9.0,1,0.0,0.0,45.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [34]:
df_test = X_test.copy()
df_test['income'] = y_pred

In [35]:
def get_aif_metrics(attr, df):

  privileged_groups = [{attr: 1}]
  unprivileged_groups = [{attr: 0}]

  attributes_params = dict(protected_attribute_names=[attr], label_names=['income'])
  dt = BinaryLabelDataset(df=df, **attributes_params)

  metric = BinaryLabelDatasetMetric(dt,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

  return metric

In [36]:
metric_sex_test = get_aif_metrics('sex', df_test)
print(f'Test Statistical Parity Difference (sex): {metric_sex_test.mean_difference()}')
print(f'Test Disparate Impact (sex): {metric_sex_test.disparate_impact()}')

Test Statistical Parity Difference (sex): -0.17033771845504564
Test Disparate Impact (sex): 0.3080170084050281


In [37]:
metric_race_test = get_aif_metrics('White', df_test)
print(f'Test Statistical Parity Difference (race) : {metric_race_test.mean_difference()}')  
print(f'Test Disparate Imapct (race): {metric_race_test.disparate_impact()}')


Test Statistical Parity Difference (race) : -0.07310620725254871
Test Disparate Imapct (race): 0.6394016549968173


In [54]:
def get_aif_metrics_2(attr, df, label_names):

  privileged_groups = [{attr: 1}]
  unprivileged_groups = [{attr: 0}]

  attributes_params = dict(protected_attribute_names=[attr], label_names=['income'])
  dt = BinaryLabelDataset(df=df, **attributes_params)

  metric = BinaryLabelDatasetMetric(dt,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

  return metric

,education-num,sex,capital-gain,capital-loss,hours-per-week,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia,income
28441,9.0,0,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
33791,9.0,1,0.0,1579.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
21441,13.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
9154,14.0,0,0.0,1902.0,45.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
23791,10.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6456,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
11196,11.0,0,0.0,1564.0,72.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
25904,12.0,0,0.0,0.0,40.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
35322,13.0,1,0.0,0.0,50.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [39]:
dt_train.instance_weights

array([1., 1., 1., ..., 1., 1., 1.])

### Pregunta 4: Aplique el algoritmo de Reweighing sobre los datos de entrenamiento. Compare las métricas de Fairness para los datos de entrenamiento antes y después de aplicar Reweighing. Luego, entrene un modelo de Regresión Logística utilizando los datos de entrenamiento modificados y obtenga las métricas de Fairness de las predicciones de dicho modelo. ¿Cómo varió el accuracy del modelo respecto al original y las métricas de fairness? (2 ptos).

#### Indicaciones adicionales:

- Considere 'Black' como grupo no privilegiado para la raza, 'White' como grupo privilegiado para la raza.
- Considere 'Female' como grupo no privilegiado para el sexo, 'Male' como grupo privilegiado para el sexo.


In [40]:
df_train

,education-num,sex,capital-gain,capital-loss,hours-per-week,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,...,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia,income
28441,9.0,0,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
33791,9.0,1,0.0,1579.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
21441,13.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
9154,14.0,0,0.0,1902.0,45.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
23791,10.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6456,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
11196,11.0,0,0.0,1564.0,72.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
25904,12.0,0,0.0,0.0,40.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
35322,13.0,1,0.0,0.0,50.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [41]:
# Aplicaremos el algoritmo de Reweighing para balancear los datos

privileged_groups = [{'White': 1}]
unprivileged_groups = [{'Black': 1}]

privileged_groups_sex= [{'sex': 1}]
unprivileged_groups_sex = [{'sex': 0}]


# Instanciamos BinaryLabelDataset con edad como atributo protegido

attributes_params = dict(protected_attribute_names=['White', 'Black'], label_names=['income'])
dt_train = BinaryLabelDataset(df=df_train, **attributes_params)

# Instanciamos Rw
rw = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

#Obtenemos los pesos

reweighted = rw.fit_transform(dt_train)



In [42]:
reweighted.instance_weights

array([1.02002944, 1.02002944, 0.9438207 , ..., 0.86305616, 0.9438207 ,
       1.02002944])

In [43]:
reweighted_metrics = BinaryLabelDatasetMetric(reweighted,
                                              unprivileged_groups=unprivileged_groups,
                                              privileged_groups=privileged_groups)


In [44]:
print(f'Reweighted Train Statistical Parity Difference (age): {reweighted_metrics.mean_difference()}')

Reweighted Train Statistical Parity Difference (age): -2.7755575615628914e-17


In [45]:
# Regresion Logistica
lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908)

# Agregamos los pesos calculados por el algoritmo de reweighing
lr.fit(X_train, np.ravel(y_train), sample_weight=reweighted.instance_weights)

LogisticRegressionCV(cv=10, random_state=908, solver='liblinear')

Metrica modelo entrenado con datos "reparados"

In [46]:
y_pred_proba = lr.predict_proba(X_test)[:, 1]
y_pred = y_pred_proba >= 0.5

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1: {f1_score(y_test, y_pred)}")

Accuracy: 0.8438916528468767
Recall: 0.5720680393912265
F1: 0.6441532258064516


Sesgo del modelo entrenado con datos "reparados"

In [47]:
df_test_rw = X_test.copy()
df_test_rw['income'] = y_pred


Discriminacion por sexo

In [48]:
metric_sex_test_rw = get_aif_metrics('sex', df_test_rw)
print(f'Reweighted: Test Statistical Parity Difference (sex): {metric_sex_test_rw.mean_difference()}')
print(f'Reweighted: Test Disparate Impact (sex): {metric_sex_test_rw.disparate_impact()}')

Reweighted: Test Statistical Parity Difference (sex): -0.15777283421676597
Reweighted: Test Disparate Impact (sex): 0.34704857164507086


In [49]:
metric_sex_test_rw = get_aif_metrics('White', df_test_rw)
print(f'Reweighted: Test Statistical Parity Difference (race): {metric_sex_test_rw.mean_difference()}')
print(f'Reweighted: Test Disparate Impact (race): {metric_sex_test_rw.disparate_impact()}')

Reweighted: Test Statistical Parity Difference (race): -0.03283219340129909
Reweighted: Test Disparate Impact (race): 0.8328405606657907


Para sex, vario de  de Test Statistical Parity Difference (sex): -0.17033771845504564  y Test Disparate Impact (sex): 0.3080170084050281
Reweighted: Test Statistical Parity Difference (sex): -0.15777283421676597 y Reweighted: Test Disparate Impact (sex): 0.34704857164507086

Para race vario de:
Test Statistical Parity Difference (race) : -0.07310620725254871 y Test Disparate Imapct (race): 0.6394016549968173 a 
Reweighted: Test Statistical Parity Difference (race): -0.03283219340129909 y Reweighted: Test Disparate Impact (race): 0.8328405606657907
